# **Hyper Parameter Tuning the ANN Model**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')

In [3]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
dataset.shape

(10000, 14)

In [5]:
dataset.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [6]:
# Separating dependent and independent features
X = dataset.iloc[:, 3:13]
y = dataset.iloc[:, 13]

In [7]:
# Converting Categorical Features to numeric (using one hot encoding)
geography = pd.get_dummies(X["Geography"], drop_first=True)
gender = pd.get_dummies(X['Gender'], drop_first=True)

In [8]:
# Concatenate the Data Frames
X = pd.concat([X, geography, gender],axis=1)

In [9]:
# Drop Unnecessary columns
X = X.drop(['Geography','Gender'],axis=1)

In [10]:
X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0


In [11]:
# Train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [12]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Performing Hyper Parameter Tuning

In [13]:
# Importing the Keras libraries and packages
import keras

In [14]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [15]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, PReLU, ELU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid

## Creating the model

In [16]:
X_train.shape

(8000, 11)

In [17]:
X_train.shape[1]

11

In [18]:
def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        
        if i==0:  ## This loop is for 1st input layer 
            model.add(Dense(nodes, input_dim = X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
            
        else:   ## This loop is for all the hidden the layers (except the last hidden layer)
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
            
    model.add(Dense(units = 1, kernel_initializer= 'glorot_uniform', activation = 'sigmoid')) # ## This is for all last layer
    
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [19]:
model = KerasClassifier(build_fn = create_model, verbose = 0)

In [20]:
layers = [[20], [40, 20], [45, 30, 15]]  # this says in first trail I am considering only 1 layer with 20 hidden neurons
            # this says in second trail I am considering only 2 layers with 40 hidden neurons in 1st layes and 20 neurons in 2nd layers
# this says in third trail I am considering only 3 layers with 45 hidden neurons in 1st layes and 30 neurons in 2nd layers, 15 neurons in 3rd layers
activations = ['sigmoid', 'relu']        

In [21]:
param_grid = dict(layers = layers, activation = activations, batch_size = [128, 256], epochs = [30])

In [22]:
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = 5)

In [23]:
grid_result = grid.fit(X_train, y_train)

In [24]:
[grid_result.best_score_, grid_result.best_params_]

[0.8572500109672546,
 {'activation': 'relu',
  'batch_size': 128,
  'epochs': 30,
  'layers': [45, 30, 15]}]

In [25]:
pred_y = grid.predict(X_test)
y_pred = (pred_y > 0.5)
y_pred

c:\users\manoj\appdata\local\programs\python\python38\lib\site-packages\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [26]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1522,   73],
       [ 203,  202]], dtype=int64)

In [27]:
from sklearn.metrics import accuracy_score
acc_score = accuracy_score(y_pred,y_test)
acc_score

0.862

By Hyper Parameter Tuning we can achieve the 86% accuracy for the nn model